In [1]:
import pandas as pd

df = pd.read_csv('fake.csv/000.part', engine='python', error_bad_lines=False, keep_default_na=False)

df.head()

,type,url,content,title,authors
0,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...,Sean Martin
1,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ..."
2,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ..."
3,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei..."
4,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,..."


In [2]:
from fastai.text.all import *

dls_lm = DataBlock(blocks=TextBlock.from_df('content', is_lm=True),
                   get_x=ColReader('text'), splitter=RandomSplitter(0.1)
                  ).dataloaders(df, bs=32, seq_len=80)

dls_lm.show_batch(max_n=2)

/userhome/cs/vicw0ng/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,"xxbos xxmaj join xxup fsm and stay informed . xxmaj get your daily xxmaj security xxmaj update delivered each day to your e - mail . xxbos xxmaj matt xxmaj drudge , editor and founder of the influential xxmaj drudge xxmaj report , took an apparent swipe at former xxmaj white xxmaj house chief strategist xxmaj steve xxmaj bannon for backing xxup u.s . xxmaj senate candidate xxmaj roy xxmaj moore , r - ala . , whose campaign is","xxmaj join xxup fsm and stay informed . xxmaj get your daily xxmaj security xxmaj update delivered each day to your e - mail . xxbos xxmaj matt xxmaj drudge , editor and founder of the influential xxmaj drudge xxmaj report , took an apparent swipe at former xxmaj white xxmaj house chief strategist xxmaj steve xxmaj bannon for backing xxup u.s . xxmaj senate candidate xxmaj roy xxmaj moore , r - ala . , whose campaign is engulfed"
1,"for doing more . xxmaj it will never be enough for the market . xxmaj our models will always tell us that we are helping the economy , and i will probably always feel that those benefits are overestimated . xxmaj and we will be able to tell ourselves that market function is not impaired and that inflation expectations are under control . xxmaj what is to stop us , other than much faster economic growth , which it is","doing more . xxmaj it will never be enough for the market . xxmaj our models will always tell us that we are helping the economy , and i will probably always feel that those benefits are overestimated . xxmaj and we will be able to tell ourselves that market function is not impaired and that inflation expectations are under control . xxmaj what is to stop us , other than much faster economic growth , which it is probably"


In [3]:
learn = language_model_learner(
        dls_lm, AWD_LSTM, drop_mult=0.3,
        metrics=[accuracy, Perplexity()]).to_fp16()

In [5]:
# learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.915998,3.757433,0.347351,42.838306,08:58


In [6]:
# learn.save('1epoch')

Path('models/1epoch.pth')

In [6]:
# learn = learn.load('1epoch')
# learn.unfreeze()

In [7]:
# learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.517755,3.473242,0.369883,32.241100,09:43
1,3.543658,3.539965,0.366176,34.465710,09:47
2,3.434362,3.529346,0.371195,34.101658,09:38
3,3.357048,3.493074,0.377160,32.886902,09:42
4,3.184052,3.447661,0.385416,31.426811,09:47
5,3.088798,3.406446,0.393076,30.157858,09:47
6,2.971667,3.378779,0.398499,29.334929,09:41
7,2.785853,3.356011,0.403602,28.674570,09:38
8,2.702044,3.349814,0.406020,28.497444,09:41
9,2.685668,3.355438,0.406083,28.658161,09:41


In [8]:
# learn.save('11epoch')

Path('models/11epoch.pth')

In [4]:
learn = learn.load('11epoch')
learn.unfreeze()

In [5]:
learn.save_encoder('finetuned')

In [6]:
dls_clas = DataBlock(blocks=(TextBlock.from_df('content'),CategoryBlock),
                     get_x=ColReader('text'), get_y=ColReader('type'), splitter=RandomSplitter(0.1)
                    ).dataloaders(df, bs=32, seq_len=80)

dls_clas.show_batch(max_n=3)

/userhome/cs/vicw0ng/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,category
0,"xxbos 2017 xxmaj year in xxmaj review : xxmaj when xxmaj things xxmaj started xxmaj getting xxmaj downright xxmaj weird – xxmaj david xxmaj collum – ( part 2 ) \n\n xxmaj headline : xxmaj bitcoin & xxmaj blockchain xxmaj searches xxmaj exceed xxmaj trump ! xxmaj blockchain xxmaj stocks xxmaj are xxmaj next ! \n\n xxmaj by xxmaj david xxmaj collum / xxmaj peak xxmaj prosperity \n\n xxmaj if you ’ve not yet read xxmaj part 1 , 2017 xxmaj year xxmaj in xxmaj review : xxmaj markets xxmaj fiddle xxmaj while xxmaj rome xxmaj burns to do so . xxmaj the whole enchilada can be downloaded as a single xxup pdf here or viewed in parts via the hot - linked contents as follows : \n\n xxmaj contents \n\n xxmaj part 1 \n\n xxmaj part 2 \n\n a warning for those who proceed xxmaj part xxmaj two is",fake
1,"xxbos xxmaj most people ca n’t resist getting the details on the latest conspiracy theories , no matter how far - fetched they may seem . xxmaj at the same time , many people quickly denounce any conspiracy theory as untrue … and sometimes as unpatriotic or just plain ridiculous . xxmaj lets not forget all of the thousands of conspiracies out of xxmaj wall xxmaj street like xxmaj bernie xxmaj madoff and many others to commit fraud and extortion , among many crimes of conspiracy . xxup usa xxmaj today reports that over 75 % of personal ads in the paper and on craigslist are married couples posing as single for a one night affair . xxmaj when someone knocks on your door to sell you a set of knives or phone cards , anything for that matter , do they have a profit motive ? xxmaj what is",unknown
2,"xxbos xxmaj weapons of xxmaj mass xxmaj migration : xxmaj interview with xxmaj michael xxmaj springmann on xxmaj europe ’s xxmaj migrant xxmaj crisis – xxmaj video \n\n xxmaj headline : xxmaj bitcoin & xxmaj blockchain xxmaj searches xxmaj exceed xxmaj trump ! xxmaj blockchain xxmaj stocks xxmaj are xxmaj next ! \n\n xxmaj published on xxmaj oct 22 , 2017 \n\n xxmaj europe ’s migrant crisis has been the biggest in modern history , prompting stark divisions along political lines . xxmaj what prompted the crisis ? xxmaj where is it going ? xxmaj today on the xxmaj truth xxmaj perspective we welcome back ex - diplomat xxup j. xxmaj michael xxmaj springmann to talk about his new book , xxmaj goodbye , xxmaj europe ? xxmaj hello , xxmaj chaos ? : xxmaj merkel ’s xxmaj migrant xxmaj bomb . xxmaj springmann argues that there ’s more to",fake


In [7]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [8]:
learn = learn.load_encoder('finetuned')

In [9]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.628790,0.543213,0.846915,01:23


In [10]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.520186,0.405315,0.876618,01:39


In [11]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.390426,0.334031,0.897182,02:01


In [12]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.315966,0.324922,0.897944,02:27
1,0.237817,0.318096,0.896420,02:26


In [13]:
learn.save('clas1')

Path('models/clas1.pth')

In [16]:
learn = learn.load('clas1')
learn.unfreeze()

In [18]:
learn.show_results()

,text,category,category_
0,"xxbos 2017 xxmaj year xxmaj in xxmaj review : xxmaj markets xxmaj fiddle xxmaj while xxmaj rome xxmaj burns \n\n xxmaj headline : xxmaj bitcoin & xxmaj blockchain xxmaj searches xxmaj exceed xxmaj trump ! xxmaj blockchain xxmaj stocks xxmaj are xxmaj next ! \n\n xxmaj by xxmaj david xxmaj collum / xxmaj peak xxmaj prosperity \n\n xxmaj every year , friend - of - the - site xxmaj david xxmaj collum writes a detailed “ year in xxmaj review ” synopsis full of keen perspective and plenty of wit . xxmaj this year ’s is no exception . xxmaj as with past years , he has graciously selected peakprosperity.com as the site where it will be published in full . xxmaj it ’s quite longer than our usual posts , but worth the time to read in full . a downloadable pdf of the full article is available here",fake,conspiracy
1,"xxbos [ gap in text ] … come in a representative capacity at the xxmaj masonic xxmaj temple . xxmaj so we were seated in about the fourth row back . xxmaj the xxmaj temple was crowded : there were people in the balcony ; there were people lined up along the walls ; there were people in all the corridors ; there were people jammed outside and across the street — all up and down the street . xxmaj it was as crowded as any building could possibly have been . xxmaj there were newsreel cameras , photographers , and reporters jammed up down in front . xxmaj there were various people seated on the platform with a speaker ’s podium in the center , and then down below the platform a huge bank of flowers extending the whole length of the platform , and then the casket draped",political,unreliable
2,"xxbos xxmaj the xxmaj government xxmaj has xxmaj been xxmaj meddling in xxmaj food and xxmaj nutrition for a xxmaj long xxmaj time \n\n xxmaj headline : xxmaj bitcoin & xxmaj blockchain xxmaj searches xxmaj exceed xxmaj trump ! xxmaj blockchain xxmaj stocks xxmaj are xxmaj next ! \n\n xxmaj government intrusion is often obvious . xxmaj we know when government taxes our income , stops us from using our drug of choice , or when they kick down our door and throw us in a cage . xxmaj but sometimes , government actions are more subtle and confusing . xxmaj it is often tempting to blame industry alone for the failures in the market and to ignore the substantial – but often less visible – role that government plays in regulating different markets . xxmaj from the housing crisis and its relationship to banking to healthcare and sky -",fake,fake
3,"xxbos mi - gov : xxmaj new xxmaj poll xxmaj gives xxmaj gretchen xxmaj whitmer ( d ) a 7 xxmaj point xxmaj lead xxmaj over xxmaj trumper xxmaj bill xxmaj schuette ( r ) xxmaj latest news today out of xxmaj michigan : xxmaj democrat gubernatorial candidate xxmaj gretchen xxmaj whitmer leads xxmaj republican xxmaj bill xxmaj schuette by seven percentage points in a hypothetical general election match - up , according to a poll of 600 likely xxmaj michigan voters . … poopdogcomedy 01 / 24 / 2018 13 26 - - \n\n xxup wi , xxup tn , xxup oh , xxup fl , xxup nv , xxup az , xxup mi & mt - sen : xxmaj james xxmaj mackler xxmaj launches xxup pac xxmaj to xxmaj win a xxmaj dem xxmaj majority xxmaj veteran and xxmaj army xxmaj attorney xxmaj james xxmaj mackler ( d.",political,political
4,"xxbos xxmaj the xxmaj xxunk xxmaj invasion , xxmaj they ’re xxmaj here \n\n xxmaj headline : xxmaj bitcoin & xxmaj blockchain xxmaj searches xxmaj exceed xxmaj trump ! xxmaj blockchain xxmaj stocks xxmaj are xxmaj next ! \n\n xxmaj dear infected and afflicted , \n\n xxmaj we are xxmaj the xxmaj dropouts . xxmaj we are xxmaj ephraim xxmaj gregory xxmaj nicholas xxmaj romanov xxmaj rasputin xxmaj xxunk , fourth generation descendant of the formerly secret xxmaj romanov / xxmaj rasputin bloodline and xxmaj colonel xxmaj ryan xxup j. xxmaj arlington , former xxmaj engineer for a xxup u.s.a . xxmaj army unit based in a secret deep underground military base , the location of which will remain secret , for no

In [20]:
df1 = pd.read_csv('fake.csv/001.part', engine='python', error_bad_lines=False, keep_default_na=False)

df1.tail()

,type,url,content,title,authors
14886,political,http://americannewsx.com/hot-off-the-press/carlson-sues-ales-sexual-harassment/,"Longtime Fox News anchor Gretchen Carlson, who was fired last month, filed a lawsuit on July 6, 2016 in the Superior Court of New Jersey against Chairman and CEO Roger Ailes for sexual harassment.\n\nThe lawsuit, which was first reported by the New York Times, alleges that Ailes\n\n“has unlawfully retaliated against Carlson and sabotaged her career because she refused his sexual advances and complained about severe and pervasive sexual harassment. Ailes retaliated against Carlson in various ways…including by terminating her employment on June 23, 2016, and, prior thereto, by, among other t...",Gretchen Carlson Sues Fox CEO Roger Ailes for Sexual Harassment,
14887,political,http://americannewsx.com/hot-off-the-press/heres-mansplaining-abortion-cannot-done/,"There are few subjects that gnaw in my craw quite like when a man tries “mansplaining” how they are somehow victims of abortion and unwanted pregnancies. So I’m here to set the records straight.\n\nIn a recent thread, a man (who, for privacy’s sake, will remain nameless) explains a detailed scenario of how things usually go awry for the “menz” and their lack of say regarding abortion.\n\n“We were together a while. Exclusive. Moved from condoms to pills. And then one day, you felt something slipping between us. We talked about splitting up, but didn’t yet. You skipped a few pills and never ...",Here’s Why “Mansplaining” Abortion Cannot Be Done,"About Dawn Emerson, Dawn Emerson Is A Mother Of Two, Happily Married To The Funniest Guy She Knows. When Not Raging Against The Republican Machine, She Creates Graphic Artwork For Several Websites."
14888,political,http://americannewsx.com/human-interest/notorious-rbgs-pre-emptive-strike-future-undue-burdens-women/,"On June 27, the Supreme Court struck down abortion restrictions in Texas in Whole Woman’s Health v. Hellerstedt. Conservative folks everywhere are of course clamoring about how heinous that is for women and the unborn. These are the same folk who claim to be for limited government, while fighting so hard to invade each and every woman’s health care choices. The female body is certainly one area where conservatives will fight tooth and nail to get the government involved in more regulation. They aren’t so anti-big government after all! At least not when it allows them to maintain patriarcha...",Supreme Court: Notorious RBG’s Pre-emptive Strike Against Future Undue Burdens For Women,"About Matthew Silvan, Matthew Silvan Is A Gay Man The American South Who Has Spent Years Fighting Against The Scapegoating, Demonization Imposed On The Lgbt Community Republicans, Religious Hypocrites, Zealots. His Writing Reflects The Constant Struggle To Overcome The Inequality, Discrimination Still Rampant In America. He Is An Advocate For Diversity, Progress, With A Passion For Nature, Preserving The Environment, Who Also Tends To Approach Things In A Lighthearted Way Despite It All."
14889,political,http://americannewsx.com/hot-off-the-press/planned-parenthood-launch-voter-registration-campaign/,"melt-down North Carolina If, like me, you delighted shamelessly in last week’s epic GOPat the tiny hands of Donald Trump, or if you raised-up your normal-sized hands in celebration at the overturn of discriminatory voter laws inand Wisconsin , then Planned Parenthood’s new voter registration campaign should ignite a small spark of joy in the pit of your commie bleeding-heart.\n\nLast Friday, Planned Parenthood announced that it will soon begin offering voter registration services in 45 states through its new program, “My Vote, My Voice.” Center volunteers will assemble and man “action” tab...",Planned Parenthood To Launch Voter Registration Campaign,"About Lisa Brenner, Staff Writer, Editor, Liberal Shade Thrower."
14890,satire,http://beehivebugle.com/2015/07/10/fun-and-inspiring-youth-speaker-actually-a-total-wreck2/,"Uta

In [54]:
df1['content'].size

14891

In [53]:
count = 0
count_correct = 0
for i in range(1000):
    if df1['content'][i] == '':
        continue
    count += 1;
    if learn.predict(df1['content'][i])[0] == df1['type'][i]:
        count_correct += 1
print(count, count_correct)
print(count_correct/count)

988 914
0.9251012145748988
